[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/DeepCache-colab/blob/main/DeepCache_colab.ipynb)

In [ ]:
%cd /content
!git clone -b dev https://github.com/camenduru/DeepCache
%cd /content/DeepCache

!pip install diffusers==0.21.4 transformers accelerate gradio
!pip install -q https://download.pytorch.org/whl/cu118/xformers-0.0.22.post4%2Bcu118-cp310-cp310-manylinux2014_x86_64.whl

import sys
sys.path.append('/content/DeepCache')
# !python stable_diffusion.py --model runwayml/stable-diffusion-v1-5

from torchvision.transforms import ToPILImage
from DeepCache.pipeline_stable_diffusion import StableDiffusionPipeline
import torch

pipe = StableDiffusionPipeline.from_pretrained('runwayml/stable-diffusion-v1-5', torch_dtype=torch.float16).to("cuda:0")
pipe.enable_xformers_memory_efficient_attention()

import gradio as gr

def generate(prompt, negative_prompt, num_inference_steps, width, height):
    deepcache_output = pipe(
        prompt, negative_prompt=negative_prompt, num_inference_steps=num_inference_steps, width=width, height=height,
        cache_interval=5, cache_layer_id=0, cache_block_id=0,
        uniform=True, #pow=1.4, center=15, # only for uniform = False
        output_type='pt', return_dict=True
    ).images[0]
    to_pil = ToPILImage()
    pil_image = to_pil(deepcache_output[0])
    return pil_image

with gr.Blocks(title=f"DeepCache", css=".gradio-container {max-width: 544px !important}") as demo:
    with gr.Group():
      prompt = gr.Textbox(show_label=False, value="a toy train. macro photo. 3d game asset")
      with gr.Row():
          negative_prompt = gr.Textbox(show_label=False, value="blurry")
          button = gr.Button()
      with gr.Row():
          width = gr.Slider(minimum=256, maximum=2048, step=8, value=512, label="width")
          height = gr.Slider(minimum=256, maximum=2048, step=8, value=512, label="height")
          num_inference_steps = gr.Slider(minimum=1, maximum=150, step=1, value=50, label="num_inference_steps")
      output_images = gr.Image(
          show_label=False,
          type="pil",
          interactive=False,
          height=512,
          width=512,
          elem_id="output_image",
      )
    button.click(fn=generate, inputs=[prompt, negative_prompt, num_inference_steps, width, height], outputs=[output_images], show_progress=True)
demo.queue().launch(inline=False, share=True, debug=True)